<a href="https://colab.research.google.com/github/yashvardhanbhardwaj/DefinitelyTyped/blob/master/code/experiment_on_open_price_for_100_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sklearn as skl
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


dateparse = lambda dates: pd.datetime.strptime(dates, '%Y-%m-%d')
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/aapl.us.txt',sep=',', index_col='Date', parse_dates=['Date'], date_parser=dateparse).fillna(0)
#plot close price
#plt.figure(figsize=(10,6))
# plt.grid(True)
# plt.xlabel('Year')
# plt.ylabel('Close Prices')
# All months data of 2017 for Close price
#data.tail()
#plt.title('All months data of 2017 for Close price')
#plt.plot(data.loc['2017-01-01':'2017-12-31']['Close'])
#plt.figure()
## Difference of close values for each date
#plt.title('Difference of close values for each date')
#plt.plot(np.diff(data.loc['2017-01-01':'2017-12-31']['Close']))
#plt.figure()
## Histogram for values
#plt.title('Histogram for values of Close for each date')
#plt.hist(np.diff(data.loc['2017-01-01':'2017-12-31']['Close']))
## plt.title('Closing Price')
## plt.show()
#plt.figure(figsize=(10,6))
## plt.grid(True)
## plt.xlabel('Year')
## plt.ylabel('Close Prices')
## All close date in file
#plt.title('All Close prices in file')
#plt.plot(data['Close'])
#plt.figure()
## Difference of Close prices
#plt.title('Difference of Close prices in file')
#plt.plot(np.diff(data['Close']))
#plt.figure()
## Histogram for whole data
#plt.title('Histogram for whole Close prices in file')
#plt.hist(np.diff(data['Close']))
#plt.figure()
#df_close = data['Close']
#df_close.plot(style='k.')
#plt.title('Scatter plot of closing price')
#plt.show()
##data.shape
## train, test = train_test_split(data, test_size=0.33)
#data.head()
df1=data.reset_index()['Open']
#df1
#plt.plot(df1)
scaler=MinMaxScaler(feature_range=(0,1))
df1=scaler.fit_transform(np.array(df1).reshape(-1,1))
#print(df1)
training_size=int(len(df1)*0.67)
test_size=len(df1)-training_size
train_data,test_data=df1[0:training_size,:],df1[training_size:len(df1),:1]
training_size,test_size

# convert an array of values into a dataset matrix
def create_dataset(dataset, time_step=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-time_step-1):
		a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100 
		dataX.append(a)
		dataY.append(dataset[i + time_step, 0])
	return np.array(dataX), np.array(dataY)
 
 
# reshape into X=t,t+1,t+2,t+3 and Y=t+4
time_step = 100
X_train, y_train = create_dataset(train_data, time_step)
X_test, ytest = create_dataset(test_data, time_step)
#print(X_train.shape), print(y_train.shape)

# reshape input to be [samples, time steps, features] which is required for LSTM
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

#X_train, X_test

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime instead.
  if __name__ == '__main__':


In [3]:
### Create the Stacked LSTM model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

model=Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(100,1)))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 50)           10400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 50)           20200     
_________________________________________________________________
lstm_2 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense (Dense)                (None, 1)                 51        
Total params: 50,851
Trainable params: 50,851
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,ytest),epochs=100,batch_size=64,verbose=1)

Epoch 1/100
86/86 [==============================] - 23s 208ms/step - loss: 5.8692e-05 - val_loss: 0.0025
Epoch 2/100
86/86 [==============================] - 16s 191ms/step - loss: 2.4805e-06 - val_loss: 0.0036
Epoch 3/100
86/86 [==============================] - 16s 191ms/step - loss: 1.9951e-06 - val_loss: 0.0032
Epoch 4/100
86/86 [==============================] - 16s 191ms/step - loss: 1.9112e-06 - val_loss: 0.0015
Epoch 5/100
86/86 [==============================] - 16s 189ms/step - loss: 1.7520e-06 - val_loss: 0.0041
Epoch 6/100
86/86 [==============================] - 17s 195ms/step - loss: 1.8804e-06 - val_loss: 0.0023
Epoch 7/100
86/86 [==============================] - 17s 194ms/step - loss: 1.9264e-06 - val_loss: 0.0021
Epoch 8/100
86/86 [==============================] - 17s 194ms/step - loss: 1.3849e-06 - val_loss: 0.0021
Epoch 9/100
86/86 [==============================] - 17s 194ms/step - loss: 1.2040e-06 - val_loss: 9.1612e-04
Epoch 10/100
86/86 [======================

In [ ]:
import tensorflow as tf
### prediction and check performance metrics
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)

In [ ]:
##Transformback to original form
train_predict=scaler.inverse_transform(train_predict)
test_predict=scaler.inverse_transform(test_predict)

In [ ]:
import math 
from sklearn.metrics import mean_squared_error
math.sqrt(mean_squared_error(y_train,train_predict))

In [ ]:
### Test Data RMSE
math.sqrt(mean_squared_error(ytest,test_predict))

In [ ]:
### Plotting 
# shift train predictions for plotting
look_back=100
trainPredictPlot = np.empty_like(df1)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
# shift test predictions for plotting
testPredictPlot = np.empty_like(df1)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(df1)-1, :] = test_predict
# plot baseline and predictions
plt.plot(scaler.inverse_transform(df1))
plt.plot(trainPredictPlot)
#plt.figure()
plt.plot(testPredictPlot)
#plt.figure()
plt.show()

In [ ]:
len(test_data)


In [ ]:
x_input=test_data[(len(test_data) - 100):].reshape(1,-1)
x_input.shape

In [ ]:
temp_input=list(x_input)
temp_input=temp_input[0].tolist()

In [ ]:
# demonstrate prediction for next 10 days
from numpy import array

lst_output=[]
n_steps=100
i=0
while(i<30):
    
    if(len(temp_input)>100):
        #print(temp_input)
        x_input=np.array(temp_input[1:])
        print("{} day input {}".format(i,x_input))
        x_input=x_input.reshape(1,-1)
        x_input = x_input.reshape((1, n_steps, 1))
        #print(x_input)
        yhat = model.predict(x_input, verbose=0)
        print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        #print(temp_input)
        lst_output.extend(yhat.tolist())
        i=i+1
    else:
        x_input = x_input.reshape((1, n_steps,1))
        yhat = model.predict(x_input, verbose=0)
        print(yhat[0])
        temp_input.extend(yhat[0].tolist())
        print(len(temp_input))
        lst_output.extend(yhat.tolist())
        i=i+1
  
print(lst_output)

In [ ]:
day_new=np.arange(1,101)
day_pred=np.arange(101,131)

In [ ]:
len(df1)

In [ ]:
plt.plot(day_new,scaler.inverse_transform(df1[8264:]))
plt.plot(day_pred,scaler.inverse_transform(lst_output))

In [ ]:
df3=df1.tolist()
df3.extend(lst_output)
plt.plot(df3[1200:])

In [ ]:

df3=scaler.inverse_transform(df3).tolist()

In [ ]:
plt.plot(df3)
